<a href="https://colab.research.google.com/github/honey799/FMML/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


# This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


# ANSWERS:
1)Yes, averaging the validation accuracy across multiple splits can give more consistent and reliable results when evaluating the performance of a machine learning model. This technique is often referred to as cross-validation, and it helps address the issue of variability in model performance that can occur due to the randomness in the data split.

Here's how it works:

 DATA SPLITTING : You divide your dataset into multiple subsets or folds. The typical choice is k-fold cross-validation, where you split the data into k equally-sized folds.

TRAINING AND VALIDITY: You train your model k times, each time using k-1 folds for training and the remaining fold for validation. This process ensures that every data point is used for validation exactly once.

AVERAGING: After running k iterations, you average the validation accuracy (or other performance metrics) from each iteration to obtain a more stable estimate of your model's performance.

By using cross-validation and averaging the results, you reduce the risk of your model's performance being overly influenced by the particular way the data is initially split. This can provide a more realistic assessment of how well your model generalizes to unseen data.

However, it's important to note that cross-validation can be computationally expensive, especially for large datasets or complex models, as it involves training and evaluating the model multiple times. Additionally, the choice of k (the number of folds) can impact the trade-off between computational cost and the reliability of the results. Common choices for k include 5-fold and 10-fold cross-validation, but the optimal value may vary depending on the dataset and problem at hand.

In summary, averaging the validation accuracy across multiple splits using cross-validation is a valuable technique to obtain more consistent and trustworthy estimates of your model's performance.



2)In machine learning, the goal is to create models that can make accurate predictions or classifications on unseen data. The estimate of how well a machine learning model will perform on unseen data is typically assessed using various evaluation metrics, and one of the most common metrics is accuracy.

To assess the accuracy of a machine learning model, you typically split your dataset into two parts: a training set and a test set. The model is trained on the training set and then evaluated on the test set. The accuracy on the test set is used as an estimate of how well the model will perform on new, unseen data.

The accuracy of this estimate depends on several factors, including:

Size and Representativeness of the Data: A larger and more representative test set tends to provide a more accurate estimate of model performance.

Complexity of the Model: If the model is too simple (underfitting) or too complex (overfitting), it may not generalize well to the test data, leading to inaccurate accuracy estimates.

Cross-Validation: In situations where the dataset is small, k-fold cross-validation can provide a more robust estimate of model performance by averaging accuracy scores over multiple train-test splits.

Data Preprocessing: Proper preprocessing of the data, such as feature scaling, handling missing values, and encoding categorical variables, can also affect the accuracy estimate.

Selection of Evaluation Metrics: Accuracy is just one of many evaluation metrics, and the choice of metric should be appropriate for the specific problem you are trying to solve. For imbalanced datasets, other metrics like precision, recall, F1-score, or area under the ROC curve (AUC-ROC) may be more informative.

In summary, to get a more accurate estimate of test accuracy in machine learning, it's important to use appropriate evaluation techniques, ensure a representative test set, avoid overfitting, and consider other evaluation metrics besides accuracy when appropriate for your problem.







4)The effect of the number of iterations on an estimate depends on the specific algorithm or method being used for estimation. In many numerical estimation processes, increasing the number of iterations can lead to a more accurate estimate, but this relationship is not always linear or guaranteed. Here are some general considerations:

Convergence: In many iterative numerical methods, increasing the number of iterations can help the algorithm converge to a more accurate solution. Convergence means that the estimate gets closer and closer to the true value with each iteration. However, some methods may converge quickly initially and then show diminishing returns in accuracy with additional iterations.

Accuracy: Generally, more iterations tend to result in a more accurate estimate. This is especially true for iterative methods used in numerical analysis and optimization, such as the Newton-Raphson method or gradient descent. Increasing the number of iterations allows the algorithm to make finer adjustments and reduce errors.

Computational Cost: While increasing iterations can improve accuracy, it also comes at a computational cost. More iterations mean more computational time and resources. There is often a trade-off between the accuracy of the estimate and the computational resources required.

Convergence Criteria: The number of iterations needed may also depend on the convergence criteria set for the algorithm. Some algorithms stop iterating when a certain level of accuracy is achieved, so if you set a higher accuracy requirement, you may need more iterations.

Dimensionality: In high-dimensional spaces or complex optimization problems, the relationship between the number of iterations and the quality of the estimate can be more complex. Some problems may require a large number of iterations to converge effectively.

In summary, while increasing the number of iterations can generally lead to a better estimate, it's essential to consider the specific algorithm, convergence properties, and computational resources available. Sometimes, a balance needs to be struck between the desired accuracy and the time/resources available for computation. Additionally, in some cases, more iterations may not significantly improve the estimate if the algorithm has already converged to a satisfactory level of accuracy.




